In [30]:
# Add parent directory to path for local imports
import os,sys,inspect
currentdir = os.path.dirname(os.path.abspath(inspect.getfile(inspect.currentframe())))
parentdir = os.path.dirname(currentdir)
sys.path.insert(0,parentdir) 

from PIL import Image
import numpy as np
import matplotlib.pyplot as plt
import os
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import torchvision
import torchvision.datasets as dset
import torchvision.models as models
import torchvision.transforms as T
import bcolz
import time
from torch.utils.data import Dataset, DataLoader, sampler
from sklearn.metrics import *
%matplotlib inline

In [2]:
use_gpu = torch.cuda.is_available()
print('Using gpu: %s ' % use_gpu)

def gpu(x,use_gpu=use_gpu):
    if use_gpu:
        return x.cuda()
    else:
        return x

Using gpu: True 


## Data processing

In [13]:
transform = T.Compose([
    T.CenterCrop(224),
    T.ToTensor(),
    T.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
])

class ImageFolderWithPaths(dset.ImageFolder):
    """Custom dataset that includes image file paths. Extends
    torchvision.datasets.ImageFolder
    """
    # override the __getitem__ method. this is the method dataloader calls
    def __getitem__(self, index):
        # this is what ImageFolder normally returns 
        original_tuple = super(ImageFolderWithPaths, self).__getitem__(index)
        # the image file path
        path = self.imgs[index][0]
        # make a new tuple that includes original and the path
        tuple_with_path = (original_tuple + (path,))
        return tuple_with_path

train_dataset = ImageFolderWithPaths('../Dataset/train', transform=transform)
val_dataset = ImageFolderWithPaths('../Dataset/val', transform=transform)
test_dataset = ImageFolderWithPaths('../Dataset/test', transform=transform)

In [14]:
train_size = len(train_dataset)
val_size = len(val_dataset)
test_size = len(test_dataset)
print("Number of training examples {}, validation examples {}, testing examples {}".format(train_size, val_size, test_size))

Number of training examples 66071, validation examples 11016, testing examples 33154


In [15]:
train_dataloader = DataLoader(train_dataset, batch_size=64, shuffle=True, num_workers=6)
val_dataloader = DataLoader(val_dataset, batch_size=64, shuffle=False, num_workers=6)
test_dataloader = DataLoader(test_dataset, batch_size=64, shuffle=False, num_workers=6)

In [16]:
IngreLabel = open('../VireoFood172/SplitAndIngreLabel/IngreLabel.txt', 'r').read().split('\n')[:-1]  # list of str
path_to_ingredients = dict()
for i in range(len(IngreLabel)):
    path_and_ingredients = IngreLabel[i].split()
    path, ingredients = path_and_ingredients[0], [0 if int(label) == -1 else int(label) for label in path_and_ingredients[1:]]
    path_to_ingredients[path] = np.array(ingredients)

In [17]:
def get_ingredients(path):
    split_path = path.split("/")
    key = '/' + '/'.join(split_path[3:])
    ingredients = path_to_ingredients[key]
    return ingredients

## Ensemble Model

In [25]:
class FoodEnsemble(nn.Module):
    def __init__(self, resnet, densenet):
        super(FoodEnsemble, self).__init__()
        self.resnet = resnet
        self.densenet = densenet
        self.alpha = 0.5
    def forward(self, x):
        with torch.no_grad():
            a1 = self.resnet(x)
            a2 = self.densenet(x)
        return self.alpha * a1 + (1 - self.alpha) * a2
        

resnet = gpu(torch.load("../saved_models/ingredients/res18/res18.dat"))
resnext = gpu(torch.load("../saved_models/ingredients/resnext/resnext.dat"))
model = FoodEnsemble(resnet, resnext)

print(model)

FoodEnsemble(
  (resnet): ResNet(
    (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
    (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (relu): ReLU(inplace=True)
    (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
    (layer1): Sequential(
      (0): BasicBlock(
        (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (relu): ReLU(inplace=True)
        (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      )
      (1): BasicBlock(
        (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track

In [26]:
# Use threshold to define predicted labels and invoke sklearn's metrics with different averaging strategies.
def calculate_metrics(pred, target, threshold=0.5):
    pred = np.array(pred > threshold, dtype=float)
    return {'micro/precision': precision_score(y_true=target, y_pred=pred, average='micro'),
            'micro/recall': recall_score(y_true=target, y_pred=pred, average='micro'),
            'micro/f1': f1_score(y_true=target, y_pred=pred, average='micro'),
            'macro/precision': precision_score(y_true=target, y_pred=pred, average='macro'),
            'macro/recall': recall_score(y_true=target, y_pred=pred, average='macro'),
            'macro/f1': f1_score(y_true=target, y_pred=pred, average='macro'),
            'samples/precision': precision_score(y_true=target, y_pred=pred, average='samples'),
            'samples/recall': recall_score(y_true=target, y_pred=pred, average='samples'),
            'samples/f1': f1_score(y_true=target, y_pred=pred, average='samples'),
            }
def get_num_correct(scores, y):
    scores[scores >= 0.5] = 1
    scores[scores < 0.5] = 0
    num_correct = (scores == y).all(dim=1).sum().item()
    return num_correct

In [34]:
criterion = nn.BCELoss()

def check_accuracy(model):
    num_correct, num_samples, total_loss = 0, 0, 0
    model.eval()
    batches = val_dataloader
    with torch.no_grad():
        model_result = []
        targets = []
        for x, food_labels, paths in batches:
            y = np.array([get_ingredients(path) for path in paths])
            x, y = gpu(x), gpu(torch.from_numpy(y)).to(torch.float32)
            scores = model(x)
            loss = criterion(scores, y) 
            total_loss += loss.data.item()
            num_correct += get_num_correct(scores, y)
            num_samples += x.size(0)
            
            model_result.extend(scores.cpu().numpy())
            targets.extend(y.cpu().numpy())
        result = calculate_metrics(np.array(model_result), np.array(targets))
        micro_f1, macro_f1, samples_f1 = result['micro/f1'], result['macro/f1'], result['samples/f1']
        print("Validation: "
              "Micro F1: {:.3f} "
              "Macro F1: {:.3f} "
              "Samples F1: {:.3f}".format(result['micro/f1'],
                                          result['macro/f1'],
                                          result['samples/f1']))
        average_loss = total_loss / num_samples
        acc = num_correct / num_samples
    print('Validation Loss: {:.8f} Got {} / {} correct {:.2f}%'.format(average_loss, num_correct, num_samples, 100 * acc))

In [35]:
%%time
check_accuracy(model)

/opt/conda/lib/python3.7/site-packages/torch/utils/data/dataloader.py:477: UserWarning: This DataLoader will create 6 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))
/opt/conda/lib/python3.7/site-packages/sklearn/metrics/_classification.py:1245: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/opt/conda/lib/python3.7/site-packages/sklearn/metrics/_classification.py:1245: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf

Validation: Micro F1: 0.722 Macro F1: 0.518 Samples F1: 0.689
Validation Loss: 0.00026477 Got 3767 / 11016 correct 34.20%
CPU times: user 1min 37s, sys: 31.2 s, total: 2min 8s
Wall time: 2min 11s


## Evaluation

In [33]:
# Evaluation: Compute Accuracy and Micro F1, Macro F1, Samples F1 scores

def compute_top_1_accuracy(model):
    num_correct, num_samples, total_loss = 0, 0, 0
    with torch.no_grad():
        model.eval()
        model_result = []
        targets = []
        for x, food_labels, paths in test_dataloader:
            y = np.array([get_ingredients(path) for path in paths])
            x, y = gpu(x), gpu(torch.from_numpy(y)).to(torch.float32)
            scores = model(x)
            loss = criterion(scores, y) 
            total_loss += loss.data.item()
            num_correct += get_num_correct(scores, y)
            num_samples += x.size(0)
            
            model_result.extend(scores.cpu().numpy())
            targets.extend(y.cpu().numpy())
        result = calculate_metrics(np.array(model_result), np.array(targets))
        micro_f1, macro_f1, samples_f1 = result['micro/f1'], result['macro/f1'], result['samples/f1']
        print("Testing: "
              "Micro F1: {:.3f} "
              "Macro F1: {:.3f} "
              "Samples F1: {:.3f}".format(result['micro/f1'],
                                          result['macro/f1'],
                                          result['samples/f1']))
        average_loss = total_loss / num_samples
        acc = num_correct / num_samples

    print('Accuracy of the network on the all test images: %.2f%%' % (
        100 * num_correct / num_samples))

compute_top_1_accuracy(model)

/opt/conda/lib/python3.7/site-packages/torch/utils/data/dataloader.py:477: UserWarning: This DataLoader will create 6 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))
/opt/conda/lib/python3.7/site-packages/sklearn/metrics/_classification.py:1245: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/opt/conda/lib/python3.7/site-packages/sklearn/metrics/_classification.py:1245: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in samples with no predicted labels. Use `zero_division` parameter to control this behavior.
  _

Testing: Micro F1: 0.718 Macro F1: 0.527 Samples F1: 0.683
Accuracy of the network on the all test images: 34.48%
